In [1]:
import sys, os
pardir = os.path.abspath(os.path.join(os.path.dirname('__file__'), os.path.pardir))
sys.path.append(pardir)

import numpy as np
import librosa
import librosa.display
from scipy.io import wavfile
from aubio import onset
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import dsp
import configobj
from validate import Validator

from read_labels import read_labels

%matplotlib inline

ModuleNotFoundError: No module named 'dsp'

In [ ]:
rootdir = '/home/tracek/Data/Birdman/'
filename = os.path.join(rootdir, 'raw/STHELENA-02_20140605_200000_1.wav')
outdir = os.path.join(rootdir, 'raw/samples/')
sheets = read_labels('/home/tracek/Data/Birdman/labels/sthelena_labels.xls')
filename_noext = os.path.splitext(os.path.basename(filename))[0]

# in seconds [s]
signal_start_s = 0
signal_end_s = 95

sheet = sheets[filename_noext]
condition = (sheet['Time Start'] > signal_start_s) & (sheet['Time End'] < signal_end_s)
sheet_selected = sheet[condition]

nfft = 256 # samples
hop = nfft // 2

In [ ]:
y, sr = librosa.load(filename, sr=None)
y = y[signal_start_s * sr: signal_end_s * sr]
S = np.abs(librosa.stft(y, n_fft=nfft, hop_length=hop))
D = librosa.amplitude_to_db(S)
D.shape

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(111)

librosa.display.specshow(D[25:,:], sr=sr, hop_length=hop, y_axis='linear', x_axis='time')

for index, row in sheet_selected.iterrows():
    if row['Species'] == 'Storm Petrel':
        ax.axvspan(xmin=row['Time Start'], xmax=row['Time End'], color='green', alpha=0.3)
    else:
        ax.axvspan(xmin=row['Time Start'], xmax=row['Time End'], color='white', alpha=0.3)

In [ ]:
yh = dsp.highpass_filter(y, sr, highcut=500)

In [ ]:
oenv = librosa.onset.onset_strength(y=yh, sr=sr, aggregate=np.median, fmax=4000, n_mels=100, lag=5)

In [ ]:
ons = librosa.onset.onset_detect(onset_envelope=oenv, sr=sr, hop_length=hop, backtrack=False, units='samples')

In [ ]:
ons_back = librosa.onset.onset_detect(onset_envelope=oenv, sr=sr, hop_length=hop, backtrack=True, units='samples')

In [ ]:
len(ons)

In [ ]:
config = configobj.ConfigObj(pardir + '/config.ini', configspec=pardir + '/configspec.ini')
validation_successful = config.validate(Validator())

onsets_forward = dsp.get_onsets_config(yh, sr, config)
onsets_backward = dsp.get_onsets_config(yh[::-1], sr, config)

In [ ]:
len(onsets_forward)

In [ ]:
len(onsets_backward)

In [ ]:
onsets_backward_reversed = [signal_end_s - onset for onset in onsets_backward]

In [ ]:
onsets_forward

In [ ]:
onsets_backward_reversed[::-1]